In [1]:
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "05_create_dataset/05_split_tfrecord.ipynb"
_nb_title = "Splitting dataset and writing TF Records"

### no need to change any of this
_nb_safeloc = _nb_loc.replace('/', '%2F')
_nb_safetitle = _nb_title.replace(' ', '+')
md("""
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name={1}&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F{2}&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F{2}">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/{0}">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
""".format(_nb_loc, _nb_safetitle, _nb_safeloc))


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Splitting+dataset+and+writing+TF+Records&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F05_create_dataset%2F05_split_tfrecord.ipynb&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F05_create_dataset%2F05_split_tfrecord.ipynb">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/05_create_dataset/05_split_tfrecord.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/05_create_dataset/05_split_tfrecord.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/05_create_dataset/05_split_tfrecord.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


# データセットの分割とTFレコードの書き込み  

このノートブックは、データセットをトレーニング、検証、テストに分割し、それらの画像をTensorFlowレコードファイルに書き込む方法を示しています。  


In [2]:
import pandas as pd
df = pd.read_csv('gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/all_data.csv', names=['image','label'])
df.head()

,image,label
0,gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/100...,daisy
1,gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/101...,daisy
2,gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/101...,daisy
3,gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/101...,daisy
4,gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/101...,daisy


In [3]:
import numpy as np
np.random.seed(10)
rnd = np.random.rand(len(df))
train = df[ rnd < 0.8  ]
valid = df[ (rnd >= 0.8) & (rnd < 0.9) ]
test  = df[ rnd >= 0.9 ]
print(len(df), len(train), len(valid), len(test))

3670 2930 359 381


In [4]:
%%bash
rm -rf output
mkdir output

In [5]:
train.to_csv('output/train.csv', header=False, index=False)
valid.to_csv('output/valid.csv', header=False, index=False)
test.to_csv('output/test.csv', header=False, index=False)

In [6]:
!head output/test.csv

gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10466290366_cc72e33532.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10712722853_5632165b04.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/11642632_1e7627a2cc.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/13583238844_573df2de8e_m.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/1374193928_a52320eafa.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/13953307149_f8de6a768c_m.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/14471433500_cdaa22e3ea_m.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/14523675369_97c31d0b5b.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/163978992_8128b49d3e_n.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/16401288243_36112bd52f_m.jpg,daisy


## ApacheBeamを使用したTFレコードの書き込み  

速度を上げるために、5つのレコードだけを書き込む方法を説明します

In [7]:
outdf = test.head()
len(outdf)

5

In [8]:
outdf.values

array([['gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10466290366_cc72e33532.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10712722853_5632165b04.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/11642632_1e7627a2cc.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/13583238844_573df2de8e_m.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/1374193928_a52320eafa.jpg',
        'daisy']], dtype=object)

In [9]:
!gsutil cat gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dict.txt

daisy
dandelion
roses
sunflowers
tulips


In [10]:
import tensorflow as tf
with tf.io.gfile.GFile('gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dict.txt', 'r') as f:
    LABELS = [line.rstrip() for line in f]
print('Read in {} labels, from {} to {}'.format(
    len(LABELS), LABELS[0], LABELS[-1]))

Read in 5 labels, from daisy to tulips


In [18]:
import apache_beam as beam
import tensorflow as tf

def _string_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def read_and_decode(filename):
    IMG_CHANNELS = 3
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

def create_tfrecord(filename, label, label_int):
    print(filename)
    img = read_and_decode(filename)
    dims = img.shape
    img = tf.reshape(img, [-1]) # flatten to 1D array
    return tf.train.Example(features=tf.train.Features(feature={
        'image': _float_feature(img),
        'shape': _int64_feature([dims[0], dims[1], dims[2]]),
        'label': _string_feature(label),
        'label_int': _int64_feature([label_int])
    })).SerializeToString()

with beam.Pipeline() as p:
    (p 
     | 'input_df' >> beam.Create(outdf.values)
     | 'create_tfrecord' >> beam.Map(lambda x: create_tfrecord(x[0], x[1], LABELS.index(x[1])))
     | 'write' >> beam.io.tfrecordio.WriteToTFRecord('output/train')
    )

gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10466290366_cc72e33532.jpg
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10712722853_5632165b04.jpg
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/11642632_1e7627a2cc.jpg
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/13583238844_573df2de8e_m.jpg
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/1374193928_a52320eafa.jpg


In [47]:
!ls -l output/train*

-rw-r--r-- 1 jupyter jupyter 8777320 Dec 25 19:58 output/train-00000-of-00001
-rw-r--r-- 1 jupyter jupyter  236472 Dec 25 19:35 output/train.csv


In [48]:
## splitting in Apache Beam
def hardcoded(x, desired_split):
    split, rec = x
    print('hardcoded: ', split, rec, desired_split, split == desired_split)
    if split == desired_split:
        yield rec

with beam.Pipeline() as p:
        splits = (p
                  | 'input_df' >> beam.Create([
                      ('train', 'a'),
                      ('train', 'b'),
                      ('valid', 'c'),
                      ('valid', 'd')
                  ]))
        
        split = 'train'
        _ = (splits
                 | 'h_only_{}'.format(split) >> beam.FlatMap(
                     lambda x: hardcoded(x, 'train'))
         )        
        split = 'valid'
        _ = (splits
                 | 'h_only_{}'.format(split) >> beam.FlatMap(
                     lambda x: hardcoded(x, 'valid'))
        )

hardcoded:  train a valid False
hardcoded:  train a train True
hardcoded:  train b valid False
hardcoded:  train b train True
hardcoded:  valid c valid True
hardcoded:  valid c train False
hardcoded:  valid d valid True
hardcoded:  valid d train False


## Dataflowで実行  

Apache Beamコードは、CloudDataflowを使用してサーバーレスで実行できます。

重要なことは次のとおりです。  
<br/>  
beam.Pipeline()を次のように置き換えます。  

```
options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': JOBNAME,
      'project': PROJECT,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'save_main_session': True
  }
opts = beam.pipeline.PipelineOptions(flags=[], **options)
with beam.Pipeline(RUNNER, options=opts) as p:
```

In [ ]:
%%bash
PROJECT=$(gcloud config get-value project)
BUCKET=${PROJECT}

python3 -m jpeg_to_tfrecord \
       --all_data gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/all_data.csv \
       --labels_file gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dict.txt \
       --project_id $PROJECT \
       --output_dir gs://${BUCKET}/data/flower_tfrecords

<img src="dataflow_pipeline.png" width="75%"/>

In [50]:
!gsutil ls -l gs://ai-analytics-solutions/data/flower_tfrecords/*-00001-*

   4475228  2020-12-25T22:58:29Z  gs://ai-analytics-solutions/data/flower_tfrecords/test-00001-of-00099
  16718085  2020-12-25T22:58:29Z  gs://ai-analytics-solutions/data/flower_tfrecords/train-00001-of-00168
   2919806  2020-12-25T22:58:29Z  gs://ai-analytics-solutions/data/flower_tfrecords/valid-00001-of-00096
TOTAL: 3 objects, 24113119 bytes (23 MiB)


Copyright 2020 Google Inc. Apache Licenseバージョン2.0(「ライセンス」)に基づいてライセンス供与されます。ライセンスに準拠する場合を除き、このファイルを使用することはできません。ライセンスのコピーはhttp://www.apache.org/licenses/LICENSE-2.0で入手できます。適用法で要求されているか、書面で合意されていない限り、ライセンスに基づいて配布されるソフトウェアは「現状有姿」で配布されます。明示または黙示を問わず、いかなる種類の保証または条件もありません。ライセンスに基づく許可と制限を規定する特定の言語については、ライセンスを参照してください。